In [ ]:
import jax
import jax.numpy as jnp
from jax import make_jaxpr

from genjax import gen, normal, pjax

In [ ]:
@gen
def model(v):
    x = normal(v, 1.0) @ "x"


type(model)

In [ ]:
jax.vmap(model.simulate, in_axes=(0,))((jnp.ones(10),))

In [ ]:
make_jaxpr(jax.vmap(model.simulate, in_axes=(0,)))((jnp.ones(10),))

In [ ]:
make_jaxpr(pjax.vmap(model.simulate, in_axes=(0,)))((jnp.ones(10),))